In [2]:
from __future__ import annotations

import math
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

import random
from pathlib import Path
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
# import matplotlib.pyplot as plt
# csv_path = "data/daily/id01/2024-06-29.csv"
# df = pd.read_csv(csv_path)

# # x축: 0~1439 (분), y축: m_charging(0/1)
# x = range(len(df))  # 1440 가정
# y = df["heart_rate"].astype(int)  # 혹시 실수/문자여도 정수로

# plt.figure(figsize=(12, 3))  # 가로:세로 ≈ 8:1 (원하신 1:6~1:8 범위)
# # 0/1 변화가 뚜렷하게 보이도록 계단형 선 스타일
# plt.plot(x, y, drawstyle="steps-post", linewidth=1.5)

# plt.title("heart_rate over 1440 minutes")
# plt.xlabel("Minute of day")
# plt.ylabel("heart_rate")
# plt.ylim(50, 120)
# plt.xlim(0, len(df)-1)
# plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.5)

# plt.tight_layout()
# plt.show()

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 2_000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float)
            * (-math.log(10_000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe, persistent=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, T, D)
        """
        return x + self.pe[: x.size(1)].unsqueeze(0)

class LocalSegmentEncoder(nn.Module):
    def __init__(self, input_dim: int, d_model: int, nhead: int, num_layers: int, dropout: float = 0.1):
        super().__init__()
        self.input_dim = input_dim
        self.per_channel = 8  # 센서당 출력 채널 수

        # 센서별 Conv (독립 처리)
        self.sensor_convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(1, self.per_channel, kernel_size=3, padding=1),
                nn.ReLU(),
            )
            for _ in range(input_dim)
        ])

        # concat 후 전체 projection: (B, input_dim * per_channel, L) → (B, d_model, L)
        self.project = nn.Sequential(
            nn.Conv1d(input_dim * self.per_channel, d_model, kernel_size=1),
            nn.ReLU(),
        )

        self.pos_enc = PositionalEncoding(d_model)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=nhead,
                dim_feedforward=d_model * 4,
                dropout=dropout,
                batch_first=True,
                norm_first=True,
            ),
            num_layers=num_layers
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, L, input_dim)
        """
        x = x.permute(0, 2, 1)  # → (B, input_dim, L)
        conv_outputs = []

        # 각 센서별로 (B, 1, L) → (B, per_channel, L)
        for i in range(self.input_dim):
            sensor_i = x[:, i:i+1, :]                   # (B, 1, L)
            out_i = self.sensor_convs[i](sensor_i)     # (B, per_channel, L)
            conv_outputs.append(out_i)

        # concat: (B, input_dim * per_channel, L)
        x = torch.cat(conv_outputs, dim=1)

        # project: (B, d_model, L)
        x = self.project(x)

        # Transformer: (B, L, d_model)
        x = x.permute(0, 2, 1)  # (B, L, d_model)
        x = self.pos_enc(x)
        return self.encoder(x)  # (B, L, d_model)

class AttentionPooling(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(dim, dim // 2),
            nn.Tanh(),
            nn.Linear(dim // 2, 1)
        )

    def forward(self, x):
        # x: (B, T, D)
        score = self.attn(x)                # (B, T, 1)
        weights = torch.softmax(score, dim=1)
        return (x * weights).sum(dim=1)     # (B, D)

class MultiLabelHTTMS(nn.Module):
    def __init__(self, segment_indices, num_users, label_configs,
                 local_d_model=256,
                 global_n_head=8, global_num_layers=5, dropout=0.1):
        super().__init__()
        self.segment_indices = segment_indices
        self.label_configs = label_configs  # e.g. [("Q1", 2), ..., ("S3", 2)]

        self.embeddings = nn.ModuleList([
            nn.Embedding(9, 4),    # m_activity
            nn.Embedding(254, 8),  # m_ambience
        ])
        self.num_cont_sensors = 13
        self.total_cat_emb_dim = sum(e.embedding_dim for e in self.embeddings)
        self.num_sensors = self.num_cont_sensors + self.total_cat_emb_dim

        self.local_encoder = LocalSegmentEncoder(
            input_dim=self.num_sensors,
            d_model=local_d_model,
            nhead=4,
            num_layers=3,
            dropout=dropout,
        )

        self.segment_pos = PositionalEncoding(local_d_model, max_len=2048)
        self.global_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=local_d_model, nhead=global_n_head,
                dim_feedforward=local_d_model * 4, dropout=dropout,
                batch_first=True, norm_first=True,
            ),
            num_layers=global_num_layers,
            enable_nested_tensor=False,
        )

        self.attn_pool = AttentionPooling(local_d_model)
        self.id_embed = nn.Embedding(num_users, 256)

        # 🔹 멀티헤드 분기
        self.heads = nn.ModuleDict({
            name: nn.Sequential(
                nn.LayerNorm(local_d_model + 256),
                nn.Linear(local_d_model + 256, 512),
                nn.GELU(),
                nn.Dropout(0.2),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(256, num_classes)
            )
            for name, num_classes in label_configs
        })

    def _split_segments(self, x: torch.Tensor) -> list:
        return [x[..., s:e] for s, e in self.segment_indices]

    def forward(self, x_cont: torch.Tensor, x_cat: torch.Tensor, user_ids: torch.Tensor) -> dict:
        """
        x_cont: (B, 13, T)
        x_cat : (B, 2,  T)
        """
        B, _, T = x_cont.shape
        x_cont = torch.nan_to_num(x_cont)  # NaN/Inf → finite

        x_cat = torch.nan_to_num(x_cat.float(), nan=0.0).long()
        for i, emb in enumerate(self.embeddings):
            xmax = emb.num_embeddings - 1
            x_cat[:, i].clamp_(0, xmax)

        cat_embs = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]   # list of (B, T, D_i)
        cat_feat = torch.cat([e.permute(0, 2, 1) for e in cat_embs], dim=1)      # (B, D_total, T)

        x = torch.cat([x_cont, cat_feat], dim=1)                                 # (B, total_sensors, T)

        segment_tensors = [seg.permute(0, 2, 1) for seg in self._split_segments(x)]  # each (B, L_seg, C)
        local_outputs = [self.local_encoder(seg) for seg in segment_tensors]         # each (B, L_seg, d_model)

        seq = torch.cat(local_outputs, dim=1)     # (B, L_total, d_model)
        seq = self.segment_pos(seq)
        seq = self.global_encoder(seq)            # (B, L_total, d_model)
        pooled = self.attn_pool(seq)              # (B, d_model)

        id_vec = self.id_embed(user_ids)          # (B, 256)
        out = torch.cat([id_vec, pooled], dim=-1) # (B, d_model + 256)

        return {name: head(out) for name, head in self.heads.items()}

In [5]:
# # =========================
# # BiLSTM Baseline
# # =========================
# class BiLSTMMultiLabel(nn.Module):
#     def __init__(self, segment_indices, num_users, label_configs,
#                  d_model=256, lstm_hidden=256, num_layers=2, dropout=0.1):
#         super().__init__()
#         self.label_configs = label_configs
#         # 범주형 임베딩 (HTTMS와 동일)
#         self.embeddings = nn.ModuleList([
#             nn.Embedding(9, 4),     # m_activity
#             nn.Embedding(254, 8),   # m_ambience
#         ])
#         self.num_cont_sensors = 6
#         self.total_cat_emb_dim = sum(e.embedding_dim for e in self.embeddings)  # 12
#         self.input_feat = self.num_cont_sensors + self.total_cat_emb_dim        # 18

#         # per-timestep feature -> d_model
#         self.proj = nn.Linear(self.input_feat, d_model)
#         self.lstm = nn.LSTM(
#             input_size=d_model, hidden_size=lstm_hidden, num_layers=num_layers,
#             batch_first=True, bidirectional=True, dropout=dropout if num_layers > 1 else 0.0
#         )
#         self.attn_pool = AttentionPooling(lstm_hidden * 2)
#         # self.id_embed = nn.Embedding(num_users, 256)

#         in_dim = lstm_hidden * 2
#         self.heads = nn.ModuleDict({
#             name: nn.Sequential(
#                 nn.LayerNorm(in_dim),
#                 nn.Linear(in_dim, 512), nn.GELU(), nn.Dropout(0.2),
#                 nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.1),
#                 nn.Linear(256, num_classes)
#             )
#             for name, num_classes in label_configs
#         })

#     def forward(self, x_cont, x_cat, user_ids):
#         B, _, T = x_cont.shape
#         x_cont = torch.nan_to_num(x_cont)

#         # categorical embeddings
#         x_cat = torch.nan_to_num(x_cat.float(), nan=0.0).long()
#         for i, emb in enumerate(self.embeddings):
#             xmax = emb.num_embeddings - 1
#             x_cat[:, i].clamp_(0, xmax)
#         cat_embs = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]   # (B,T,D_i)
#         cat_feat = torch.cat([e.permute(0,2,1) for e in cat_embs], dim=1)        # (B, D_total, T)

#         x = torch.cat([x_cont, cat_feat], dim=1)                                 # (B, 18, T)
#         x = x.permute(0,2,1)                                                     # (B, T, 18)
#         x = self.proj(x)                                                         # (B, T, d_model)

#         y, _ = self.lstm(x)                                                      # (B, T, 2*hidden)
#         pooled = self.attn_pool(y)                                               # (B, 2*hidden)

#         # id_vec = self.id_embed(user_ids)                                         # (B, 256)
#         # rep = torch.cat([pooled, id_vec], dim=-1)

#         return {name: head(pooled) for name, head in self.heads.items()}


# # =========================
# # 1D-CNN Baseline
# # =========================
# class CNN1DMultiLabel(nn.Module):
#     def __init__(self, segment_indices, num_users, label_configs,
#                  stem_channels=128, blocks=4, dropout=0.1, downsample_stride=2):
#         super().__init__()
#         self.label_configs = label_configs
#         self.embeddings = nn.ModuleList([
#             nn.Embedding(9, 4),
#             nn.Embedding(254, 8),
#         ])
#         self.num_cont_sensors = 6
#         self.total_cat_emb_dim = sum(e.embedding_dim for e in self.embeddings)
#         in_ch = self.num_cont_sensors + self.total_cat_emb_dim  # 18

#         layers = [nn.Conv1d(in_ch, stem_channels, kernel_size=7, padding=3),
#                   nn.ReLU(), nn.Dropout(dropout)]
#         ch = stem_channels
#         for b in range(blocks):
#             layers += [
#                 nn.Conv1d(ch, ch, kernel_size=3, padding=1),
#                 nn.ReLU(),
#                 nn.Conv1d(ch, ch, kernel_size=3, padding=1),
#                 nn.ReLU(),
#             ]
#             # 다운샘플로 길이 줄여 Transformer 대비 공정성 ↑
#             layers += [nn.Conv1d(ch, ch, kernel_size=3, stride=downsample_stride, padding=1),
#                        nn.ReLU()]
#         self.cnn = nn.Sequential(*layers)

#         self.attn_pool = AttentionPooling(ch)
#         # self.id_embed = nn.Embedding(num_users, 256)

#         in_dim = ch
#         self.heads = nn.ModuleDict({
#             name: nn.Sequential(
#                 nn.LayerNorm(in_dim),
#                 nn.Linear(in_dim, 512), nn.GELU(), nn.Dropout(0.2),
#                 nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.1),
#                 nn.Linear(256, num_classes)
#             )
#             for name, num_classes in label_configs
#         })

#     def forward(self, x_cont, x_cat, user_ids):
#         x_cont = torch.nan_to_num(x_cont)

#         x_cat = torch.nan_to_num(x_cat.float(), nan=0.0).long()
#         for i, emb in enumerate(self.embeddings):
#             xmax = emb.num_embeddings - 1
#             x_cat[:, i].clamp_(0, xmax)
#         cat_embs = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
#         cat_feat = torch.cat([e.permute(0,2,1) for e in cat_embs], dim=1)        # (B, 12, T)

#         x = torch.cat([x_cont, cat_feat], dim=1)                                 # (B, 18, T)
#         feat = self.cnn(x)                                                       # (B, C, T')
#         seq = feat.permute(0, 2, 1)                                              # (B, T', C)

#         pooled = self.attn_pool(seq)                                             # (B, C)
#         # id_vec = self.id_embed(user_ids)                                         # (B, 256)
#         # rep = torch.cat([pooled, id_vec], dim=-1)

#         return {name: head(pooled) for name, head in self.heads.items()}


# # =========================
# # Vanilla Transformer Baseline
# # =========================
# class VanillaTransformerMultiLabel(nn.Module):
#     def __init__(self, segment_indices, num_users, label_configs,
#                  d_model=256, nhead=8, num_layers=4, dropout=0.1, patch_stride=4):
#         super().__init__()
#         self.label_configs = label_configs
#         self.embeddings = nn.ModuleList([
#             nn.Embedding(9, 4),
#             nn.Embedding(254, 8),
#         ])
#         self.num_cont_sensors = 6
#         self.total_cat_emb_dim = sum(e.embedding_dim for e in self.embeddings)
#         in_ch = self.num_cont_sensors + self.total_cat_emb_dim  # 18

#         # 길이 1440이 길 수 있어, 가벼운 patch embedding으로 다운샘플
#         self.patch = nn.Conv1d(in_ch, d_model, kernel_size=patch_stride*2-1,
#                                stride=patch_stride, padding=patch_stride-1)
#         self.pos = PositionalEncoding(d_model, max_len=2048)
#         enc_layer = nn.TransformerEncoderLayer(
#             d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
#             dropout=dropout, batch_first=True, norm_first=True
#         )
#         self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers,
#                                              enable_nested_tensor=False)  # 경고 억제
#         self.attn_pool = AttentionPooling(d_model)
#         # self.id_embed = nn.Embedding(num_users, 256)

#         in_dim = d_model
#         self.heads = nn.ModuleDict({
#             name: nn.Sequential(
#                 nn.LayerNorm(in_dim),
#                 nn.Linear(in_dim, 512), nn.GELU(), nn.Dropout(0.2),
#                 nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.1),
#                 nn.Linear(256, num_classes)
#             )
#             for name, num_classes in label_configs
#         })

#     def forward(self, x_cont, x_cat, user_ids):
#         x_cont = torch.nan_to_num(x_cont)

#         x_cat = torch.nan_to_num(x_cat.float(), nan=0.0).long()
#         for i, emb in enumerate(self.embeddings):
#             xmax = emb.num_embeddings - 1
#             x_cat[:, i].clamp_(0, xmax)
#         cat_embs = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
#         cat_feat = torch.cat([e.permute(0,2,1) for e in cat_embs], dim=1)        # (B, 12, T)

#         x = torch.cat([x_cont, cat_feat], dim=1)                                 # (B, 18, T)
#         x = self.patch(x)                                                        # (B, d_model, T')
#         x = x.permute(0, 2, 1)                                                   # (B, T', d_model)
#         x = self.pos(x)
#         x = self.encoder(x)
#         pooled = self.attn_pool(x)

#         # id_vec = self.id_embed(user_ids)
#         # rep = torch.cat([pooled, id_vec], dim=-1)

#         return {name: head(pooled) for name, head in self.heads.items()}

In [6]:
# -----------------------------
# Dataset
# -----------------------------
class MultiLabelDataset(Dataset):
    def __init__(self, label_csv: Path, data_root: Path, label_configs,
                 cont_idx=None, cat_idx=None, zero_out_cats=False):
        self.df = pd.read_csv(label_csv)
        self.rows = self.df.to_dict(orient="records")
        self.data_root = data_root
        self.label_names = [name for name, _ in label_configs]

        # --- 센서 타입 고정 (sensor_order 인덱스 기준) ---
        # 7,8: categorical / 11,12,14: binary / others: continuous
        default_cat_idx = [7, 8]
        default_bin_idx = [11, 12, 14]
        default_cont_only_idx = [i for i in range(15) if i not in default_cat_idx + default_bin_idx]  # [0, 5, 9, 10, 12, 13]


        # 외부에서 cont_idx/cat_idx 주면 우선, 아니면 기본 사용
        self.cat_idx = cat_idx if cat_idx is not None else default_cat_idx
        self.bin_idx = default_bin_idx
        # x_cont에는 "연속 10 + 이진 3"을 합쳐 총 13채널을 넣습니다.
        self.cont_only_idx = cont_idx if cont_idx is not None else default_cont_only_idx

        self.zero_out_cats = zero_out_cats
        self.user_id_dict = {f"id{str(i).zfill(2)}": i - 1 for i in range(1, 11)}

    def __getitem__(self, idx):
        r = self.rows[idx]
        csv_path = self.data_root / "daily" / r["subject_id"] / f"{r['sleep_date']}.csv"
        x = torch.from_numpy(pd.read_csv(csv_path).values.T.astype("float32"))  # (S=15, T)
        x = torch.nan_to_num(x)

        # ---- 연속(10) + 이진(3) → x_cont(13채널) ----
        x_cont_only = x[self.cont_only_idx].float()     # (10, T)
        x_bin = x[self.bin_idx].float()                 # (3, T)
        x_bin = torch.nan_to_num(x_bin, nan=0.0)
        x_bin.clamp_(0.0, 1.0)                          # 안전하게 0/1로
        x_cont = torch.cat([x_cont_only, x_bin], dim=0) # (13, T)

        # ---- 범주형(2) → 임베딩 입력 ----
        if self.zero_out_cats:
            T = x.shape[1]
            x_cat = torch.zeros((len(self.cat_idx), T), dtype=torch.long)
        else:
            x_cat = x[self.cat_idx].float()
            x_cat = torch.nan_to_num(x_cat, nan=0.0).long()
            # m_activity ∈ [0,8], m_ambience ∈ [0,253]
            if x_cat.size(0) >= 1: x_cat[0].clamp_(0, 8)
            if x_cat.size(0) >= 2: x_cat[1].clamp_(0, 253)

        labels = {name: torch.tensor(r[name], dtype=torch.long) for name in self.label_names}
        user_id = torch.tensor(self.user_id_dict[r["subject_id"]], dtype=torch.long)

        # 세그먼트(그대로)
        T = x.shape[1]
        num_segments = 3
        seg_len = T // num_segments
        seg_bounds = [(i * seg_len, (i + 1) * seg_len) for i in range(num_segments - 1)]
        seg_bounds.append(((num_segments - 1) * seg_len, T))
        return x_cont, x_cat, labels, seg_bounds, user_id

    def __len__(self):
        return len(self.rows)

def collate_fn(batch):
    x_conts, x_cats, label_dicts, bounds, user_ids = zip(*batch)
    batched_labels = {k: torch.stack([d[k] for d in label_dicts]) for k in label_dicts[0].keys()}
    return torch.stack(x_conts), torch.stack(x_cats), batched_labels, bounds[0], torch.stack(user_ids)

# -----------------------------
# Split
# -----------------------------
class SubsetDataset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
    def __getitem__(self, i):
        return self.dataset[self.indices[i]]
    def __len__(self):
        return len(self.indices)

# -----------------------------
# Utils
# -----------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def compute_class_weights(dataset: MultiLabelDataset, indices, label_name: str, num_classes: int, device):
    ys = [dataset.rows[i][label_name] for i in indices]
    cnt = np.bincount(ys, minlength=num_classes).astype(np.float32)
    cnt_clip = np.clip(cnt, 1.0, None)
    # inverse frequency 기반, 평균 1 정도가 되도록 정규화
    weights = (cnt_clip.sum() / (num_classes * cnt_clip))
    return torch.tensor(weights, dtype=torch.float32, device=device)

# -----------------------------
# Train (Multi-Head)
# -----------------------------
def train_multilabel_model_kfold(
    label_configs,
    label_weights,
    dataset: MultiLabelDataset,
    n_splits: int = 5,
    test_ratio: float = 0.1,
    stratify_label: str = "Q1",
    model_class=MultiLabelHTTMS,
    model_kwargs: dict | None = None,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")

    # 1) train/test 먼저 분리
    all_indices = list(range(len(dataset)))
    y_all = [dataset.rows[i][stratify_label] for i in all_indices]
    trainval_idx, test_idx = train_test_split(
        all_indices, test_size=test_ratio, stratify=y_all, random_state=42
    )
    y_trainval = [dataset.rows[i][stratify_label] for i in trainval_idx]

    test_loader = DataLoader(
        SubsetDataset(dataset, test_idx), batch_size=BATCH_SIZE, shuffle=False,
        collate_fn=collate_fn, num_workers=2, pin_memory=True
    )

    # 2) K-Fold on trainval
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_ckpts = []
    cv_f1_list, cv_acc_list = [], []

    for fold, (tr_rel, va_rel) in enumerate(skf.split(trainval_idx, y_trainval), start=1):
        tr_idx = [trainval_idx[i] for i in tr_rel]
        va_idx = [trainval_idx[i] for i in va_rel]

        train_loader = DataLoader(
            SubsetDataset(dataset, tr_idx), batch_size=BATCH_SIZE, shuffle=True,
            collate_fn=collate_fn, num_workers=2, pin_memory=True
        )
        val_loader = DataLoader(
            SubsetDataset(dataset, va_idx), batch_size=BATCH_SIZE, shuffle=False,
            collate_fn=collate_fn, num_workers=2, pin_memory=True
        )

        # 세그먼트 인덱스(해당 fold의 train 첫 배치 기준) 고정
        _, _, _, seg_bounds_ref, _ = next(iter(train_loader))

        # 모델 & 손실함수(클래스 가중치: fold의 train 분포로 계산)
        kwargs = dict(segment_indices=seg_bounds_ref, num_users=10, label_configs=label_configs)
        if model_kwargs: kwargs.update(model_kwargs)
        model = model_class(**kwargs).to(device)

        loss_fns = {}
        for name, num_cls in label_configs:
            w = compute_class_weights(dataset, tr_idx, name, num_cls, device)
            loss_fns[name] = nn.CrossEntropyLoss(weight=w)

        optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
        scaler = torch.amp.GradScaler(enabled=use_amp)

        denom = sum(label_weights.values())
        best_f1 = -1.0
        ckpt_path = f"pt/multilabel_best_model_fold{fold}.pt"

        for epoch in range(1, EPOCHS + 1):
            # ---- Train ----
            model.train()
            running_loss, n_samples = 0.0, 0
            pbar = tqdm(train_loader, desc=f"[Fold {fold} | Train {epoch}]")
            for x_cont, x_cat, labels, _, user_ids in pbar:
                x_cont = x_cont.to(device, non_blocking=True)
                x_cat = x_cat.to(device, non_blocking=True)
                user_ids = user_ids.to(device, non_blocking=True)
                labels = {k: v.to(device, non_blocking=True) for k, v in labels.items()}

                optimizer.zero_grad(set_to_none=True)
                with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                    outputs = model(x_cont, x_cat, user_ids)
                    loss = sum(label_weights[k] * loss_fns[k](outputs[k], labels[k]) for k in outputs) / denom

                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()

                bs = x_cont.size(0)
                running_loss += loss.item() * bs
                n_samples += bs
                pbar.set_postfix_str(f"loss={running_loss/max(n_samples,1):.4f}")

            # ---- Val ----
            model.eval()
            all_preds = {k: [] for k, _ in label_configs}
            all_trues = {k: [] for k, _ in label_configs}
            val_loss_sum, val_count = 0.0, 0

            with torch.no_grad():
                for x_cont, x_cat, labels, _, user_ids in tqdm(val_loader, desc=f"[Fold {fold} | Val {epoch}]"):
                    x_cont = x_cont.to(device, non_blocking=True)
                    x_cat = x_cat.to(device, non_blocking=True)
                    user_ids = user_ids.to(device, non_blocking=True)
                    labels = {k: v.to(device, non_blocking=True) for k, v in labels.items()}

                    with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                        outputs = model(x_cont, x_cat, user_ids)
                        batch_loss = sum(label_weights[k] * loss_fns[k](outputs[k], labels[k]) for k in outputs) / denom

                    bs = x_cont.size(0)
                    val_loss_sum += batch_loss.item() * bs
                    val_count += bs

                    for k in outputs:
                        pred = outputs[k].argmax(dim=-1)
                        all_preds[k].extend(pred.cpu().tolist())
                        all_trues[k].extend(labels[k].cpu().tolist())

            avg_loss = val_loss_sum / max(val_count, 1)
            f1s = {k: f1_score(all_trues[k], all_preds[k], average="macro") for k, _ in label_configs}
            accs = {k: accuracy_score(all_trues[k], all_preds[k]) for k, _ in label_configs}
            avg_f1 = float(np.mean(list(f1s.values())))
            avg_acc = float(np.mean(list(accs.values())))

            head_log = " | ".join([f"{k}:F1={f1s[k]:.3f}/Acc={accs[k]:.3f}" for k, _ in label_configs])
            print(f"📊 [Fold {fold}] Epoch {epoch}: ValLoss={avg_loss:.4f} | ValAcc={avg_acc:.4f} | ValMacroF1={avg_f1:.4f}")
            print("   ", head_log)

            if avg_f1 > best_f1:
                best_f1 = avg_f1
                torch.save(model.state_dict(), ckpt_path)
                print(f"✅ [Fold {fold}] Saved best: {ckpt_path} (F1={best_f1:.4f})")

        fold_ckpts.append(ckpt_path)
        cv_f1_list.append(best_f1)

    print("\n====== CV Summary ======")
    print("Fold F1s:", [f"{s:.4f}" for s in cv_f1_list])
    print(f"CV Mean F1: {np.mean(cv_f1_list):.4f}  |  Std: {np.std(cv_f1_list):.4f}")

    return fold_ckpts, test_loader

def evaluate_multilabel_model_ensemble(ckpt_paths, test_loader, label_configs,
                                       num_users=10,
                                       model_class=MultiLabelHTTMS,
                                       model_kwargs: dict | None = None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")

    # test_loader의 첫 배치에서 세그먼트 인덱스 추출 (T가 전 샘플 동일하다는 가정)
    x_cont0, x_cat0, labels0, seg_bounds_ref, user_ids0 = next(iter(test_loader))

    # 모델들 로드(메모리에 동시에 올려 앙상블)
    models = []
    for p in ckpt_paths:
        kwargs = dict(segment_indices=seg_bounds_ref, num_users=num_users, label_configs=label_configs)
        if model_kwargs: kwargs.update(model_kwargs)
        m = model_class(**kwargs).to(device)
        state = torch.load(p, map_location=device)
        m.load_state_dict(state)
        m.eval()
        models.append(m)

    # 누적 컨테이너
    all_preds = {k: [] for k, _ in label_configs}
    all_trues = {k: [] for k, _ in label_configs}

    # 평가 루프
    with torch.no_grad():
        for x_cont, x_cat, labels, _, user_ids in test_loader:
            x_cont = x_cont.to(device, non_blocking=True)
            x_cat = x_cat.to(device, non_blocking=True)
            user_ids = user_ids.to(device, non_blocking=True)
            labels = {k: v.to(device, non_blocking=True) for k, v in labels.items()}

            # fold 예측 합산(로짓 평균)
            sum_logits = None
            for m in models:
                with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                    outs = m(x_cont, x_cat, user_ids)  # dict[k] -> (B, C)
                if sum_logits is None:
                    sum_logits = {k: v.clone() for k, v in outs.items()}
                else:
                    for k in sum_logits:
                        sum_logits[k] += outs[k]

            # 평균 로짓으로 최종 예측
            for k in sum_logits:
                avg_logits = sum_logits[k] / len(models)
                pred = avg_logits.argmax(dim=1).cpu().tolist()
                true = labels[k].cpu().tolist()
                all_preds[k].extend(pred)
                all_trues[k].extend(true)

    # 지표 계산
    per_label_f1  = {k: f1_score(all_trues[k], all_preds[k], average="macro") for k, _ in label_configs}
    per_label_acc = {k: accuracy_score(all_trues[k], all_preds[k])             for k, _ in label_configs}
    macro_f1  = float(np.mean(list(per_label_f1.values())))
    macro_acc = float(np.mean(list(per_label_acc.values())))

    print("✅ Final Test (Ensemble) Results")
    print("-" * 40)
    for k in per_label_f1:
        print(f"{k}: F1={per_label_f1[k]:.4f} | Acc={per_label_acc[k]:.4f}")
    print("-" * 40)
    print(f"Macro F1={macro_f1:.4f} | Macro Acc={macro_acc:.4f}")

    return {
        "Per-label F1": per_label_f1,
        "Per-label Accuracy": per_label_acc,
        "Macro F1": macro_f1,
        "Macro Accuracy": macro_acc
    }

In [8]:
# -----------------------------
# Seed/Config/Run
# -----------------------------
set_seed(42)

DATA_DIR = Path("data")
LABEL_CSV = DATA_DIR / "ch2025_metrics_train.csv"
BATCH_SIZE = 16
EPOCHS = 20
LR = 2e-4
os.makedirs("pt", exist_ok=True)

label_configs = [("Q1", 2), ("Q2", 2), ("Q3", 2), ("S1", 3), ("S2", 2), ("S3", 2)]
label_weights = {"Q1": 1.0, "Q2": 1.0, "Q3": 1.0, "S1": 1.5, "S2": 1.0, "S3": 1.0}  # 헤드간 가중치(필요시 조정)

dataset = MultiLabelDataset(LABEL_CSV, DATA_DIR, label_configs)

# HTTMS
fold_ckpts, test_loader = train_multilabel_model_kfold(
    label_configs, label_weights, dataset,
    n_splits=5, test_ratio=0.2, stratify_label="Q1",
    model_class=MultiLabelHTTMS, model_kwargs={}
)
results = evaluate_multilabel_model_ensemble(
    fold_ckpts, test_loader, label_configs,
    model_class=MultiLabelHTTMS, model_kwargs={}
)

# # Bi-LSTM
# fold_ckpts_lstm, test_loader = train_multilabel_model_kfold(
#     label_configs, label_weights, dataset,
#     n_splits=5, test_ratio=0.2, stratify_label="Q1",
#     model_class=BiLSTMMultiLabel,
#     model_kwargs=dict(d_model=256, lstm_hidden=256, num_layers=2, dropout=0.1)
# )
# results_lstm = evaluate_multilabel_model_ensemble(
#     fold_ckpts_lstm, test_loader, label_configs,
#     model_class=BiLSTMMultiLabel,
#     model_kwargs=dict(d_model=256, lstm_hidden=256, num_layers=2, dropout=0.1)
# )

# # 1D-CNN
# fold_ckpts_cnn, test_loader = train_multilabel_model_kfold(
#     label_configs, label_weights, dataset,
#     n_splits=5, test_ratio=0.2, stratify_label="Q1",
#     model_class=CNN1DMultiLabel,
#     model_kwargs=dict(stem_channels=128, blocks=4, dropout=0.1, downsample_stride=2)
# )
# results_cnn = evaluate_multilabel_model_ensemble(
#     fold_ckpts_cnn, test_loader, label_configs,
#     model_class=CNN1DMultiLabel,
#     model_kwargs=dict(stem_channels=128, blocks=4, dropout=0.1, downsample_stride=2)
# )

# # Vanilla Transformer
# fold_ckpts_vit, test_loader = train_multilabel_model_kfold(
#     label_configs, label_weights, dataset,
#     n_splits=5, test_ratio=0.2, stratify_label="Q1",
#     model_class=VanillaTransformerMultiLabel,
#     model_kwargs=dict(d_model=256, nhead=8, num_layers=4, dropout=0.1, patch_stride=4)
# )
# results_vit = evaluate_multilabel_model_ensemble(
#     fold_ckpts_vit, test_loader, label_configs,
#     model_class=VanillaTransformerMultiLabel,
#     model_kwargs=dict(d_model=256, nhead=8, num_layers=4, dropout=0.1, patch_stride=4)
# )

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[Fold 1 | Train 1]:   0%|          | 0/18 [00:00<?, ?it/s]

[Fold 1 | Val 1]:   0%|          | 0/5 [00:00<?, ?it/s]

📊 [Fold 1] Epoch 1: ValLoss=0.7793 | ValAcc=0.4745 | ValMacroF1=0.3997
    Q1:F1=0.333/Acc=0.500 | Q2:F1=0.294/Acc=0.417 | Q3:F1=0.265/Acc=0.361 | S1:F1=0.392/Acc=0.417 | S2:F1=0.500/Acc=0.500 | S3:F1=0.613/Acc=0.653
✅ [Fold 1] Saved best: pt/multilabel_best_model_fold1.pt (F1=0.3997)


[Fold 1 | Train 2]:   0%|          | 0/18 [00:00<?, ?it/s]

...

[Fold 5 | Train 25]:   0%|          | 0/18 [00:00<?, ?it/s]

[Fold 5 | Val 25]:   0%|          | 0/5 [00:00<?, ?it/s]

📊 [Fold 5] Epoch 25: ValLoss=0.8034 | ValAcc=0.5718 | ValMacroF1=0.5508
    Q1:F1=0.575/Acc=0.583 | Q2:F1=0.522/Acc=0.528 | Q3:F1=0.405/Acc=0.417 | S1:F1=0.460/Acc=0.500 | S2:F1=0.699/Acc=0.722 | S3:F1=0.644/Acc=0.681

====== CV Summary ======
Fold F1s: ['0.5701', '0.5927', '0.6000', '0.6251', '0.6019']
CV Mean F1: 0.5980  |  Std: 0.0177


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


✅ Final Test (Ensemble) Results
----------------------------------------
Q1: F1=0.6107 | Acc=0.6111
Q2: F1=0.5398 | Acc=0.5444
Q3: F1=0.7110 | Acc=0.7111
S1: F1=0.4843 | Acc=0.5111
S2: F1=0.6627 | Acc=0.6778
S3: F1=0.6099 | Acc=0.6222
----------------------------------------
Macro F1=0.6031 | Macro Acc=0.6130


In [7]:
from pathlib import Path
from typing import Tuple
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import os

set_seed(42)

# ==== 학습 코드의 모델/모듈을 import 했다고 가정 ====
# from train_file import MultiLabelHTTMS, PositionalEncoding, LocalSegmentEncoder, AttentionPooling

USER_ID_DICT = {f"id{str(i).zfill(2)}": i - 1 for i in range(1, 11)}
DATA_DIR = Path("data/daily")
SUBMISSION_CSV = Path("data/ch2025_submission_sample.csv")
OUTPUT_PATH = Path("sub/HTTMS.csv")
os.makedirs("sub", exist_ok=True)

# 학습 때 사용한 센서 구성과 동일하게!
DEFAULT_CAT_IDX = [7, 8]       # m_activity, m_ambience
DEFAULT_BIN_IDX = [11, 12, 14] # binary 3
DEFAULT_CONT_ONLY_IDX = [i for i in range(15) if i not in DEFAULT_CAT_IDX + DEFAULT_BIN_IDX]  # 10개

# 세그먼트 개수 (학습과 동일하게 3)
NUM_SEGMENTS = 3
N_FOLDS = 5

# 라벨 구성 (학습과 동일)
label_configs = [("Q1", 2), ("Q2", 2), ("Q3", 2), ("S1", 3), ("S2", 2), ("S3", 2)]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_amp = (device.type == "cuda")

@torch.no_grad()
def load_day_like_training(csv_path: Path) -> Tuple[torch.Tensor, torch.Tensor, list]:
    """
    학습 Dataset.__getitem__과 동일 전처리:
      - 연속 10개 + 이진 3개 → x_cont (13, T)
      - 범주 2개 → x_cat (2, T)  (m_activity[0..8], m_ambience[0..253])
      - 세그먼트 3분할
    """
    df = pd.read_csv(csv_path)
    x = torch.from_numpy(df.values.T.astype("float32"))  # (15, T)
    x = torch.nan_to_num(x)

    # 연속(10) + 이진(3)
    x_cont_only = x[DEFAULT_CONT_ONLY_IDX].float()                # (10, T)
    x_bin = torch.nan_to_num(x[DEFAULT_BIN_IDX].float(), nan=0.0) # (3, T)
    x_bin.clamp_(0.0, 1.0)
    x_cont = torch.cat([x_cont_only, x_bin], dim=0)               # (13, T)

    # 범주(2)
    x_cat = torch.nan_to_num(x[DEFAULT_CAT_IDX].float(), nan=0.0).long()  # (2, T)
    # clamp to vocab range
    x_cat[0].clamp_(0, 8)     # m_activity
    x_cat[1].clamp_(0, 253)   # m_ambience

    # 세그먼트 3등분
    T = x.shape[1]
    seg_len = T // NUM_SEGMENTS
    seg_bounds = [(i * seg_len, (i + 1) * seg_len) for i in range(NUM_SEGMENTS - 1)]
    seg_bounds.append(((NUM_SEGMENTS - 1) * seg_len, T))

    # 배치 차원 추가
    return x_cont.unsqueeze(0), x_cat.unsqueeze(0), seg_bounds

# -----------------------------
# 세그먼트 기준 추출 (샘플 1개로 고정)
submission_df = pd.read_csv(SUBMISSION_CSV)
first_row = submission_df.iloc[0]
first_path = DATA_DIR / first_row["subject_id"] / f"{first_row['lifelog_date']}.csv"
_, _, seg_bounds_ref = load_day_like_training(first_path)

# -----------------------------
# 폴드별 멀티헤드 모델 로드
models = []
for fold in range(1, N_FOLDS + 1):
    m = MultiLabelHTTMS(
        segment_indices=seg_bounds_ref,
        num_users=10,
        label_configs=label_configs,
        # 학습 때와 동일 하이퍼파라미터가 있다면 여기서 지정
        # local_d_model=256, global_n_head=8, global_num_layers=5, dropout=0.1
    ).to(device)
    state = torch.load(f"pt/multilabel_best_model_fold{fold}.pt", map_location=device)
    m.load_state_dict(state)
    m.eval()
    models.append(m)

# -----------------------------
# 추론 루프
results = []
for _, row in tqdm(submission_df.iterrows(), total=len(submission_df)):
    subject = row["subject_id"]
    lifelog_date = row["lifelog_date"]
    sleep_date = row["sleep_date"]

    csv_path = DATA_DIR / subject / f"{lifelog_date}.csv"
    if not csv_path.exists():
        print(f"[WARNING] Missing file: {csv_path}")
        continue

    try:
        x_cont, x_cat, _ = load_day_like_training(csv_path)  # (1, 13, T), (1, 2, T)
        user_id = USER_ID_DICT.get(subject, 0)
        user_ids = torch.tensor([user_id], device=device)

        x_cont = x_cont.to(device)
        x_cat  = x_cat.to(device)

        # 폴드 평균 로짓 앙상블
        sum_logits = None
        with torch.no_grad():
            for m in models:
                with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                    outs = m(x_cont, x_cat, user_ids)  # dict[label] -> (1, C)
                if sum_logits is None:
                    sum_logits = {k: v.clone() for k, v in outs.items()}
                else:
                    for k in sum_logits:
                        sum_logits[k] += outs[k]

        # 최종 예측
        row_out = {
            "subject_id": subject,
            "sleep_date": sleep_date,
            "lifelog_date": lifelog_date,
        }
        for k in sum_logits:
            avg_logits = sum_logits[k] / len(models)
            pred = int(avg_logits.argmax(dim=-1).item())
            row_out[k] = pred

        results.append(row_out)

    except Exception as e:
        print(f"[ERROR] {csv_path}: {e}")

# 저장
pred_df = pd.DataFrame(results)
pred_df.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ Saved K-Fold ensemble predictions to {OUTPUT_PATH}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
100%|██████████| 250/250 [02:11<00:00,  1.90it/s]



✅ Saved K-Fold ensemble predictions to sub/HTTMS.csv
